## Problem Statement : 
The excel below contains data on the work done by AGI bots who can multi-task and do multiple types of work. They record the start and end time of each task that they undertake along with the name of the activity in this excel file.

[Time Series](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/a5d30693-0a11-417a-a01c-078ea10bea91/Untitled.xlsx)

Since the bots can multi-task and therefore can be doing multiple tasks in parallel, it is not possible to directly determine when they were working and when they were idle. You are required to:

1. Perform transformations on the data to output continuous periods (start, end) of work done by each bot and aggregate the activities done during such periods as an array against each period.
2. Solve the problem using python and SQL both

### Reading the File : 

In [1]:
import pandas as pd 
df = pd.read_excel('Time Series.xlsx')

In [6]:
df.head(10)

,Name,Start,End,Activity
0,Priyanka,2023-10-10 22:43:52.620,2023-07-20 03:31:52.620,Inspection
1,Jyoti,2023-08-24 05:55:52.620,2023-05-17 20:19:52.620,Remote Inspection
2,Jyoti,2023-06-08 08:19:52.620,2023-04-08 05:55:52.620,Updates
3,Priyanka,2023-09-21 15:31:52.620,2023-05-27 10:43:52.620,Reporting
4,Priyanka,2023-10-07 03:31:52.620,2023-04-30 13:07:52.620,Reply to Customers
5,Priyanka,2023-05-19 03:31:52.620,2023-05-11 20:19:52.620,Updates
6,Deepti,2023-08-02 13:07:52.620,2023-04-25 10:43:52.620,Fund raising
7,Jyoti,2023-05-04 22:43:52.620,2023-08-17 20:19:52.620,Business Development
8,Priyanka,2023-08-15 03:31:52.620,2023-07-02 13:07:52.620,Reporting
9,Sharan,2023-06-08 13:07:52.620,2023-09-24 01:07:52.620,Reply to Customers


In [3]:
df.isna().sum()

Name        0
Start       0
End         0
Activity    0
dtype: int64

In [4]:
df['Name'].value_counts()

Sharan      2241
Ravi        2201
Priyanka    2165
Jyoti       2140
Deepti      2138
Name: Name, dtype: int64

In [12]:
from pandasql import sqldf

In [5]:
pd.set_option('display.max_rows', None)

In [2]:
df_priyanka = df[df['Name'] == 'Priyanka']

In [3]:
df_priyanka_sorted = df_priyanka.sort_values('Start',ascending=True)
df_priyanka_sorted

,Name,Start,End,Activity
2743,Priyanka,2023-03-29 15:31:52.620,2023-05-31 15:31:52.620,Business Development
10663,Priyanka,2023-03-29 17:55:52.620,2023-06-25 17:55:52.620,Remote Inspection
4962,Priyanka,2023-03-29 22:43:52.620,2023-08-19 10:43:52.620,Podcast
904,Priyanka,2023-03-30 01:07:52.620,2023-09-01 08:19:52.620,Call
6884,Priyanka,2023-03-30 01:07:52.620,2023-06-16 01:07:52.620,Fund raising
...,...,...,...,...
1531,Priyanka,2023-10-15 03:31:52.620,2023-08-21 10:43:52.620,Call
6227,Priyanka,2023-10-15 05:55:52.620,2023-08-01 15:31:52.620,Call
766,Priyanka,2023-10-15 13:07:52.620,2023-09-11 05:55:52.620,Send Email
2013,Priyanka,2023-10-15 15:31:52.620,2023-07-29 20:19:52.620,Call


In [9]:
import numpy as np
df_priyanka_sorted.head(10)
new_index = range(len(df_priyanka_sorted))

# Set the new index on the DataFrame
df_priyanka_sorted.index = new_index

In [10]:
lag_component = []

for i in range(0,df_priyanka_sorted.shape[0]):
    lag_component.append(df.iloc[i+1]['Start'])
    
lag_component.append(np.nan)
lag_series = lag['']

df['']

In [5]:
df_priyanka_sorted

def label(row):
    if row['End'] > row['Lag_Component']:
        return 'Conti'
    else:
        return 'DisConti'
# Creating a label as 'Continous' Time Period vs 'Discontinouts Time Period'

df_priyanka_sorted['label'] = df_priyanka_sorted.apply(label,axis = 1 )

KeyError: 'Lag_Component'

In [79]:
start_time = None
end_time = None
activities = []

# Iterate over the rows
result = []
for _, row in df_priyanka_sorted.iterrows():
    if start_time is None:
        start_time = row['Start']
        end_time = row['End']
        activities.append(row['Activity'])
    elif row['label'] == 'Conti':
        end_time = row['End']
        activities.append(row['Activity'])
    else:
        result.append({
            'Start Time': start_time,
            'End Time': end_time,
            'Activities': ', '.join(activities)
        })
        start_time = None
        end_time = None
        activities = []


if start_time is not None:
    result.append({
        'Start Time': start_time,
        'End Time': end_time,
        'Activities': ', '.join(activities)
    })

# Create the resulting DataFrame
result_df = pd.DataFrame(result)

result_df_priyanka = result_df
result_df_priyanka['Name'] = 'Priyanka'
result_df_priyanka

,Start Time,End Time,Activities,Name
0,2023-03-29 15:31:52.620,2023-09-26 08:19:52.620,"Business Development, Remote Inspection, Podca...",Priyanka
1,2023-04-04 05:55:52.620,2023-05-15 20:19:52.620,"Podcast, Updates, Podcast, Reply to Customers,...",Priyanka
2,2023-04-06 08:19:52.620,2023-04-09 03:31:52.620,"Call, Fund raising, Remote Inspection, Call, U...",Priyanka
3,2023-04-09 01:07:52.620,2023-04-20 22:43:52.620,"Reply to Customers, Send Email, Podcast, Reply...",Priyanka
4,2023-04-09 17:55:52.620,2023-09-04 08:19:52.620,"Podcast, Inspection, Reporting, Updates, Send ...",Priyanka
5,2023-04-10 08:19:52.620,2023-09-14 03:31:52.620,Business Development,Priyanka
6,2023-04-10 10:43:52.620,2023-06-09 03:31:52.620,"Podcast, Reply to Customers, Updates",Priyanka
7,2023-04-10 20:19:52.620,2023-09-27 10:43:52.620,"Fund raising, Remote Inspection, Inspection, R...",Priyanka
8,2023-04-12 20:19:52.620,2023-05-13 01:07:52.620,"Inspection, Business Development, Call, Inspec...",Priyanka
9,2023-04-15 08:19:52.620,2023-04-28 03:31:52.620,"Inspection, Reporting, Call, Reporting, Podcas...",Priyanka


In [80]:
# Similary we do it for all the bots: 

all_names = df['Name'].unique()
all_names

array(['Priyanka', 'Jyoti', 'Deepti', 'Sharan', 'Ravi'], dtype=object)

In [84]:
all_names = list(all_names)
all_names.remove('Priyanka')

In [86]:
for i in 

SyntaxError: invalid syntax (<ipython-input-86-3a7d36c83a16>, line 2)